In [2]:
import cv2 
import numpy 
import skvideo.io as skv


In [3]:
def process_frame(frame_img,frame_num,start_time= None,last_frame=None,last_frame_hsv=None, threshold = 40.0):
    #resizing frame to a very low resolution
    frame_img = cv2.resize(frame_img, (80,60))
    cut_detected= False
    
    if last_frame is not None:
        #convert rgb to hsv
        hsv_curr_frame_img = cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV)
        hsv_last_frame_img = cv2.cvtColor(last_frame, cv2.COLOR_BGR2HSV)
        #split frame channels
        curr_frame_hsv =[ hsv_curr_frame_img[:, :, 0], hsv_curr_frame_img[:, :, 1], hsv_curr_frame_img[:, :, 2] ]#array of 3 channels H,S,V
        last_frame_hsv =[ hsv_last_frame_img[:, :, 0], hsv_last_frame_img[:, :, 1], hsv_last_frame_img[:, :, 2] ]

        delta_hsv_avg, delta_h, delta_s, delta_v = 0.0, 0.0, 0.0, 0.0

        #calculate change in frame hsv values
        delta_hsv = [-1, -1, -1]
        for i in range(3):
            num_pixels= curr_frame_hsv[i].shape[0] * curr_frame_hsv[i].shape[1]
            curr_frame_hsv[i] = curr_frame_hsv[i].astype(numpy.int32)
            last_frame_hsv[i] = last_frame_hsv[i].astype(numpy.int32)
            delta_hsv[i] = numpy.sum(numpy.abs(curr_frame_hsv[i] - last_frame_hsv[i])) / float(num_pixels)
        delta_hsv.append(sum(delta_hsv) / 3.0)
        delta_h, delta_s, delta_v, delta_hsv_avg = delta_hsv

        last_frame_hsv = curr_frame_hsv

        if delta_hsv_avg >= threshold:
            cut_detected=True 
            #print('cut detected ')
    
        del last_frame
    
    last_frame=frame_img.copy()
    return cut_detected
    
    
    
    
def get_cuts(cap):
    frame_num = 0
    cuts_list = list()
    frames_list = list()
    while True:
        ret, frame = cap.read()
        frame_num += 1        
        if not ret:
            break
        if process_frame(frame,frame_num):
            #list of frame numbers with cuts
            cuts_list.append(frame_num)
            frames_list.append(frame)
    
    
    print('time taken for processing = ', (cv2.getTickCount() - start_time)/cv2.getTickFrequency() , 'seconds')
    
    print ' cuts detected at frames :'
    for frame in frames_list:
        cv2.imshow('frame',frame)
        cv2.waitKey(0)
        

#a twenty second video clip 
cap = skv.VideoCapture("595_615.mp4") 
global start_time  
start_time = cv2.getTickCount()

if not cap.isOpened():
    print ('ERROR')
else:
    get_cuts(cap)

cap.release()
cv2.destroyAllWindows()

('time taken for processing = ', 13.881577149, 'seconds')


True